In [1]:
import chess
import random
import time
import torch

from backend.ai.search.random_ai import RandomAI
from backend.ai.search.alphabeta_ai import AlphaBetaAI

%load_ext autoreload

In [2]:
%autoreload

In [ ]:
board = chess.Board()
ai = RandomAI()
move_count = 1

while not board.is_game_over():
  move = ai.select_move(board)
  board.push(move)

  # 最小限の表示（前の手と簡易盤面）
  print(f"\n[{move_count}]")
  print(board)
  time.sleep(0.5)  # ← リアルタイム風に0.5秒止める
  move_count += 1

print("\nGame Over:", board.result())


In [ ]:
PIECE_VALUES = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 0
}



legal_moves = list(board.legal_moves)
random.shuffle(legal_moves)

# Evaluate all legal moves
for move in board.legal_moves:

  # Shuffle legal moves to add some randomness
  legal_moves = list(board.legal_moves)
  random.shuffle(legal_moves)

  # Evaluate all legal moves
  for move in board.legal_moves:
    print(f"Evaluating move: {move}")
    board.push(move)

    score = 0
    best_move = None
    best_score = -float('inf') if board.turn == chess.WHITE else float('inf')
    for piece in board.piece_map().values():
      value = PIECE_VALUES[piece.piece_type]
      if piece.color == chess.WHITE:
        score += value
      else:
        score -= value
        
    board.pop()
    

    if board.turn == chess.WHITE:
      if score > best_score:
        best_score = score
        best_move = move
    else:
      if score < best_score:
        best_score = score
        best_move = move

Evaluating move: g1h3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
R 5
N 3
B 3
K 0
Q 9
B 3
N 3
R 5
Evaluating move: g1f3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
R 5
N 3
B 3
K 0
Q 9
B 3
N 3
R 5
Evaluating move: b1c3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
R 5
N 3
B 3
K 0
Q 9
B 3
N 3
R 5
Evaluating move: b1a3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
R 5
N 3
B 3
K 0
Q 9
B 3
N 3
R 5
Evaluating move: h2h3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
R 5
N 3
B 3
K 0
Q 9
B 3
N 3
R 5
Evaluating move: g2g3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
P 1
R 5
N 3
B 3
K 0
Q 9
B 3
N 3
R 5
Evaluating move: f2f3
r 5
n 3
b 3
k 0
q 9
b 3
n 3
r 5
p 1
p 1
p 1
p 1
p 1
p 1
p 1
p 1
P 1
P 1
P 1
P 

In [7]:
board.legal_moves

<LegalMoveGenerator at 0x7b268c0cd1d0 (Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4)>

In [3]:
board = chess.Board()
positions = []

ai_white = AlphaBetaAI(depth=4)
ai_black = AlphaBetaAI(depth=4)

move_count = 1

while not board.is_game_over():
  tensor = board_to_tensor(board)
  positions.append((tensor.clone(), board.turn))

  move = ai_white.select_move(board) if board.turn == chess.WHITE else ai_black.select_move(board)
  if move is None:
    print("No legal moves available, game over.")
  else:
    board.push(move)

  print(f"\n[{move_count}]")
  print(board)
  time.sleep(0.25)
  move_count += 1

result = board.result()  # '1-0', '0-1', '1/2-1/2'
print("\nGame Over:", result)

if result == "1-0":
  reward_white, reward_black = 1.0, 0.0
elif result == "0-1":
  reward_white, reward_black = 0.0, 1.0
else:
  reward_white = reward_black = 0.5

data = []
for tensor, turn in positions:
  reward = reward_white if turn == chess.WHITE else reward_black
  data.append((tensor, torch.tensor([reward])))




[1]
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R

[2]
r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R

[3]
r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . P . . . .
. . . Q . . . .
P P P . P P P P
R N B . K B N R

[4]
r n b q k b n r
p p p p . p . p
. . . . p . . .
. . . . . . p .
. . . P . . . .
. . . Q . . . .
P P P . P P P P
R N B . K B N R

[5]
r n b q k b n r
p p p p . p . Q
. . . . p . . .
. . . . . . p .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B . K B N R

[6]
r n b q k b n r
p p p p . . . Q
. . . . p p . .
. . . . . . p .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B . K B N R

[7]
r n b q k b n Q
p p p p . . . .
. . . . p p . .
. . . . . . p .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B . K B N R

[8]
r n b q k b . Q
p p p p . . . .
. . . . p p . n
. . . . . . p .


In [5]:
positions

[(tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.

In [4]:
data

[(tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.

In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from backend.ai.nnue.nnue_model import SimpleNNUE  # 必要に応じてパスを調整

from backend.ai.constants import local_paths

data = torch.load(os.path.join(local_paths.DATA_SELF_LEARNING_DATASET, "self_learning_dataset_1.pt"))  # データのパスを適宜調整
print(f"データ数: {len(data)}")
print(f"1件目の型: {type(data[0])}")
print(f"1件目の要素数: {len(data[0])}")
print(f"1件目の入力テンソル shape: {data[0][0].shape}")
print(f"1件目の報酬値: {data[0][1]}")


データ数: 29154
1件目の型: <class 'tuple'>
1件目の要素数: 2
1件目の入力テンソル shape: torch.Size([768])
1件目の報酬値: tensor([1.])


In [ ]:
def apply_move_to_tensor(tensor: torch.Tensor, move: chess.Move, board: chess.Board) -> torch.Tensor:
  tensor = tensor.clone()  # 元のテンソルを壊さないようコピー

  moving_piece = board.piece_at(move.from_square)
  if moving_piece is not None:
    # 移動元のビットを 0 に
    piece_type = moving_piece.piece_type - 1
    color_offset = 0 if moving_piece.color == chess.WHITE else 6 * 64
    from_idx = color_offset + piece_type * 64 + move.from_square
    to_idx = color_offset + piece_type * 64 + move.to_square
    tensor[from_idx] = 0
    tensor[to_idx] = 1

  if board.is_capture(move):
    # キャプチャ対象のビットを 0 に
    captured_piece = board.piece_at(move.to_square)
    if captured_piece is not None:
      cap_type = captured_piece.piece_type - 1
      cap_offset = 0 if captured_piece.color == chess.WHITE else 6 * 64
      cap_idx = cap_offset + cap_type * 64 + move.to_square
      tensor[cap_idx] = 0

  return tensor


In [2]:
board = chess.Board()
opening_books = {
  chess.WHITE: ["e2e4", "d2d4", "c2c4", "g1f3"],
  chess.BLACK: ["e7e5", "c7c5", "e7e6", "g8f6"]
}


move_uci = random.choice(opening_books[board.turn])
move = chess.Move.from_uci(move_uci)

print(f"Opening move: {move_uci}")

Opening move: d2d4


In [3]:
moving_piece = board.piece_at(move.from_square)
if moving_piece is not None:
	print(moving_piece.piece_type)
else:
	print("No piece on the from_square.")

1


In [4]:
print(move.from_square, move.to_square)

11 27


In [5]:
board.is_capture(move)

False

In [6]:
board.push(move)
print(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R


In [79]:
print(board)

r q . . . . . r
p . p b p n b p
n . . k . p . .
. p . p . . p .
. P . P . . . P
N . P . Q P . R
P . . . P K . .
R . B . . B N .


In [80]:
legal_moves = list(board.legal_moves)
move = random.choice(legal_moves)
print(board.is_capture(move))
moving_piece = board.piece_at(move.from_square)
print(move.from_square, move.to_square)
if moving_piece:
  print(moving_piece.piece_type - 1)

board.push(move)

True
20 52
4


In [81]:
print(board)

r q . . . . . r
p . p b Q n b p
n . . k . p . .
. p . p . . p .
. P . P . . . P
N . P . . P . R
P . . . P K . .
R . B . . B N .


In [2]:
import torch

torch.cuda.is_available()

True

In [2]:
from backend.ai.utils.move_ordering import move_ordering_score
import chess

board = chess.Board()
legal_moves = list(board.legal_moves)
legal_moves.sort(key=lambda m: move_ordering_score(board, m), reverse=True)

In [3]:
legal_moves

[Move.from_uci('g1h3'),
 Move.from_uci('g1f3'),
 Move.from_uci('b1c3'),
 Move.from_uci('b1a3'),
 Move.from_uci('h2h3'),
 Move.from_uci('g2g3'),
 Move.from_uci('f2f3'),
 Move.from_uci('e2e3'),
 Move.from_uci('d2d3'),
 Move.from_uci('c2c3'),
 Move.from_uci('b2b3'),
 Move.from_uci('a2a3'),
 Move.from_uci('h2h4'),
 Move.from_uci('g2g4'),
 Move.from_uci('f2f4'),
 Move.from_uci('e2e4'),
 Move.from_uci('d2d4'),
 Move.from_uci('c2c4'),
 Move.from_uci('b2b4'),
 Move.from_uci('a2a4')]

In [4]:
import torch
import torch.nn as nn

# ダミーNNモデル（MLP1層）
class DummyNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(774, 1)

  def forward(self, x):
    return self.linear(x)

# 評価キャッシュクラス
class EvalCache:
  def __init__(self):
    self.cache = {}
    self.calls = 0  # 実際にNNを呼んだ回数を記録

  def tensor_to_key(self, tensor: torch.Tensor) -> int:
    return hash(tensor.detach().cpu().numpy().tobytes())

  def evaluate(self, model, tensor: torch.Tensor) -> float:
    key = self.tensor_to_key(tensor)
    if key in self.cache:
      return self.cache[key]
    self.calls += 1  # 実際に呼ばれたときだけカウント
    with torch.no_grad():
      score = model(tensor.unsqueeze(0)).item()
    self.cache[key] = score
    return score


In [5]:
# モデル・キャッシュ初期化
model = DummyNN()
cache = EvalCache()

# 同じtensorを何度も使う
tensor = torch.rand(774)  # 固定のランダム入力

# NN評価を10回試す（うち1回だけが実際に呼ばれるはず）
for i in range(10):
  score = cache.evaluate(model, tensor)

print(f"NN was actually called {cache.calls} time(s).")


NN was actually called 1 time(s).


In [20]:
from backend.ai.utils.cache.compute_zobrist import compute_zobrist_hash
board1 = chess.Board()
board2 = chess.Board()

board1.push(chess.Move.from_uci('d2d4'))
board1.push(chess.Move.from_uci('b7b6'))
board1.push(chess.Move.from_uci('d4d5'))
board1.push(chess.Move.from_uci('e7e5'))
print(board1, "\n")

board2.push(chess.Move.from_uci('d2d4'))
board2.push(chess.Move.from_uci('e7e6'))
board2.push(chess.Move.from_uci('d4d5'))
board2.push(chess.Move.from_uci('e6e5'))
board2.push(chess.Move.from_uci('g1f3'))
board2.push(chess.Move.from_uci('f3g1'))
board2.push(chess.Move.from_uci('b7b6'))

print(board2)

compute_zobrist_hash(board1) == compute_zobrist_hash(board2)

r n b q k b n r
p . p p . p p p
. p . . . . . .
. . . P p . . .
. . . . . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R 

r n b q k b n r
p . p p . p p p
. P . . . . . .
. . . P p . . .
. . . . . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B n R


False